In [58]:
# 일단 2015데이터로 학습, 예측

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import missingno as msno
import numpy  as np
import seaborn as sns
%matplotlib inline

path = './data/'
pd.options.display.max_rows = 150
pd.options.display.max_columns = 350
plt.rc('font',family='Malgun Gothic')
plt.rcParams['axes.unicode_minus'] = False

In [2]:
sido = pd.read_csv(path+'sido_imp_10.csv', encoding='cp949')

In [3]:
sido_y = sido.iloc[:,-7:]
sido_x = sido.iloc[:,3:-7]
sido_name = sido.iloc[:,:3]

### 표준화

In [4]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))
sido_x_scaled = scaler.fit_transform(sido_x)

In [5]:
sido_x_scaled = pd.DataFrame(data = sido_x_scaled)
sido_x_scaled.columns = sido_x.columns
sido_x_scaled.shape

(17, 163)

### 전체변수 넣고 다양한 알고리즘을 실험해보자

스코어나오는 함수 설정

In [47]:
#교차검증 실험 crossvalscore이용
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
cv = KFold(3, shuffle=True, random_state=0)
def KFold_meanscore(ary):
    tot = 0
    for i in ary:
        tot += i
    return tot/len(ary),ary
def kfold_crossvalscore(model, X, y, cv=cv):
    ary1 = cross_val_score(model, X, y, cv=cv, scoring='f1')
    ary2 = cross_val_score(model, X, y, cv=cv, scoring='accuracy')
    ary3 = cross_val_score(model, X, y, cv=cv, scoring='precision')
    ary4 = cross_val_score(model, X, y, cv=cv, scoring='recall')
    print(f'f1-score:{KFold_meanscore(ary1)}')
    print(f'accuracy:{KFold_meanscore(ary2)}')
    print(f'precision:{KFold_meanscore(ary3)}')
    print(f'recall:{KFold_meanscore(ary4)}')

In [48]:
#분류메트릭이용
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
def metric_score(area = sido.감염병, pred = pred):
    print(f'accuracy :\t{accuracy_score(area,pred)}')
    print(f'precision :\t{precision_score(area, pred)}')
    print(f'recall :\t{recall_score(area, pred)}')

In [56]:
#정답지 출력
def paper(y, pred=pred):
    a = pd.DataFrame(data=pred,columns=['예측'])
    b = pd.DataFrame(data=y)
    return pd.concat([a,b],axis=1)

# MLP

In [45]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(50,30))
mlp.fit(sido_x_scaled, sido.감염병)
pred = mlp.predict(sido_x_scaled)

In [59]:
mlp.score(sido_x_scaled, sido.감염병) #모델에 트레이닝셋을 적용했을때 평균정확도(accuracy) 반환

1.0

In [61]:
confusion_matrix = pd.crosstab(sido.감염병, pred, rownames=['True'], colnames=['Predicted'], margins=True)
confusion_matrix

Predicted,0,1,All
True,,,
0,11,0,11
1,0,6,6
All,11,6,17


In [46]:
#crossvalscore이용
kfold_crossvalscore(mlp, sido_x_scaled, sido.감염병)

C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


f1-score:(0.6222222222222222, array([0.4       , 0.66666667, 0.8       ]))
accuracy:(0.7666666666666666, array([0.66666667, 0.83333333, 0.8       ]))
precision:(0.4444444444444444, array([0.33333333, 0.        , 1.        ]))
recall:(0.27777777777777773, array([0.5       , 0.        , 0.33333333]))


In [43]:
#분류메트릭이용
metric_score(sido.감염병, pred)

In [49]:
#classification report
from sklearn.metrics import classification_report
print(classification_report(sido.감염병,pred, target_names=['123','45']))

              precision    recall  f1-score   support

         123       1.00      1.00      1.00        11
          45       1.00      1.00      1.00         6

    accuracy                           1.00        17
   macro avg       1.00      1.00      1.00        17
weighted avg       1.00      1.00      1.00        17



In [57]:
#정답지
paper(sido.감염병, pred)

,예측,감염병
0,0,0
1,0,0
2,0,0
3,1,1
4,0,0
5,1,1
6,0,0
7,1,1
8,0,0
9,1,1
